In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from lightning import pytorch as pl
from sklearn.model_selection import train_test_split
import torch
torch.set_float32_matmul_precision('medium')
from chemprop import data, featurizers, models, nn
pl.seed_everything(1)
from utils import *
# from data_utils import data_prep, prep_data, load_data, split_data, preprocess_data, create_data_loaders, create_mpnn_model
%load_ext autoreload
%autoreload 2

Seed set to 1
Seed set to 1


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
input_path = r"C:\Users\rhys-\OneDrive\data_hnrs\spectra\mcgill\computed_spectra.parquet" # path to your data .csv file
num_workers = 0 # number of workers for dataloader. 0 means using main process for data loading
smiles_column = "smiles" # name of the column containing SMILES strings
target_columns = np.arange(400,4002,2).astype(str) # list of names of the columns containing targets

df_input = pd.read_parquet(input_path)


,smiles,400,402,404,406,408,410,412,414,416,...,3982,3984,3986,3988,3990,3992,3994,3996,3998,4000
0,CN(c1ccc(NC(=O)Nc2ccccc2)cc1)S(=O)(=O)c1ccc(-c...,0.000771,0.000772,0.000774,0.000785,0.000792,0.000797,0.000800,0.000809,0.000812,...,0.000774,0.000786,0.000803,0.000780,0.000782,0.000776,0.000770,0.000765,0.000764,0.000767
1,CC(O)CC(C)C#COC#CC(C)CC(C)O,0.001267,0.001275,0.001275,0.001290,0.001292,0.001303,0.001306,0.001320,0.001335,...,0.001032,0.001046,0.001065,0.001040,0.001047,0.001011,0.001003,0.001020,0.001040,0.000981
2,Cc1cc([N+](=O)[O-])ccc1NC(=O)c1ccc(OCC(C)C)c(B...,0.000540,0.000540,0.000540,0.000541,0.000545,0.000547,0.000545,0.000545,0.000548,...,0.000462,0.000465,0.000453,0.000481,0.000469,0.000452,0.000458,0.000458,0.000465,0.000448
3,COC12C(COC(N)=O)C3=C(C(=O)C(C)=C(N)C3=O)N1CC1NC12,0.001041,0.001044,0.001044,0.001053,0.001056,0.001063,0.001061,0.001069,0.001074,...,0.001045,0.001055,0.001057,0.001063,0.001051,0.001031,0.001030,0.001023,0.001026,0.001022
4,CC(C)(O)C(NC(=O)c1cnn2cc(C3CC3)cnc12)c1ccc(OC(...,0.000721,0.000727,0.000726,0.000733,0.000737,0.000743,0.000745,0.000748,0.000760,...,0.000649,0.000653,0.000655,0.000656,0.000650,0.000636,0.000633,0.000632,0.000635,0.000628
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85501,N#Cc1c(Br)[nH]c2c(O)cccc12,0.001536,0.001529,0.001526,0.001542,0.001543,0.001550,0.001555,0.001582,0.001577,...,0.000386,0.000379,0.000344,0.000405,0.000378,0.000348,0.000360,0.000366,0.000375,0.000352
85502,CC(=O)c1cnc2sc(C(=O)Nc3cc(NC(=O)c4cccc(C(C)(C)...,0.001173,0.001170,0.001179,0.001178,0.001185,0.001190,0.001186,0.001191,0.001203,...,0.000725,0.000735,0.000743,0.000736,0.000736,0.000719,0.000718,0.000723,0.000732,0.000705
85503,Cc1ccc(C=NNc2cc(C(F)(F)F)ccc2Cl)cc1,0.000756,0.000757,0.000751,0.000758,0.000764,0.000762,0.000760,0.000759,0.000758,...,0.000368,0.000365,0.000349,0.000381,0.000366,0.000352,0.000358,0.000356,0.000358,0.000354
85504,CN(C)Cc1c(C(=O)NC2N=C(c3ccccc3F)c3cccc4c3N(CC4...,0.001383,0.001370,0.001379,0.001362,0.001364,0.001355,0.001344,0.001336,0.001336,...,0.000793,0.000801,0.000797,0.000811,0.000799,0.000788,0.000790,0.000780,0.000778,0.000785


In [17]:
def normalize_rows(df):
    # Select the numerical columns (excluding the first column)
    numerical_columns = df.columns[1:]
    
    # Find the maximum value in each row (excluding the first column)
    max_values = df[numerical_columns].max(axis=1)
    
    # Divide each value in the numerical columns by the corresponding maximum value
    df[numerical_columns] = df[numerical_columns].div(max_values, axis=0)
    
    return df

df_input = normalize_rows(df_input)

In [19]:
nist_test_path = r"C:\Users\rhys-\OneDrive\Documents\Github\thesis_results\data\nist\nist_test.parquet"

nist_test_df = pd.read_parquet(nist_test_path)

test_datapoints = get_mol_datapoints(nist_test_df, smiles_column, target_columns)

test_dset = data.MoleculeDataset(test_datapoints, featurizers.SimpleMoleculeMolGraphFeaturizer())

test_loader = data.MolGraphDataLoader(test_dset, num_workers=num_workers)

In [20]:
def create_rndm_loader(test_data):
    featurizer = featurizers.SimpleMoleculeMolGraphFeaturizer()


    test_dset = data.MoleculeDataset(test_data, featurizer)

    return data.MolGraphDataLoader(test_dset, num_workers=0)

# rndm_test_loader = create_rndm_loader(test_1[:700])

In [21]:
datapoints = get_mol_datapoints(df_input, smiles_column, target_columns)

train_idx, val_idx, test_idx = data.splitting.split_data(
    datapoints,
    num_folds=8,
    split='cv',
)

train_1 = [datapoints[i] for i in train_idx[0]]
val_1 = [datapoints[i] for i in val_idx[0]]
test_1 = [datapoints[i] for i in test_idx[0]]

train_2 = [datapoints[i] for i in train_idx[1]]
val_2 = [datapoints[i] for i in val_idx[1]]
test_2 = [datapoints[i] for i in test_idx[1]]

train_3 = [datapoints[i] for i in train_idx[2]]
val_3 = [datapoints[i] for i in val_idx[2]]
test_3 = [datapoints[i] for i in test_idx[2]]

train_4 = [datapoints[i] for i in train_idx[3]]
val_4 = [datapoints[i] for i in val_idx[3]]
test_4 = [datapoints[i] for i in test_idx[3]]

train_5 = [datapoints[i] for i in train_idx[4]]
val_5 = [datapoints[i] for i in val_idx[4]]
test_5 = [datapoints[i] for i in test_idx[4]]

train_6 = [datapoints[i] for i in train_idx[5]]
val_6 = [datapoints[i] for i in val_idx[5]]
test_6= [datapoints[i] for i in test_idx[5]]

train_7 = [datapoints[i] for i in train_idx[6]]
val_7 = [datapoints[i] for i in val_idx[6]]
test_7 = [datapoints[i] for i in test_idx[6]]

train_8 = [datapoints[i] for i in train_idx[7]]
val_8 = [datapoints[i] for i in val_idx[7]]
test_8 = [datapoints[i] for i in test_idx[7]]

def create_data_loaders(train_data, val_data, test_data, num_workers=0):
    featurizer = featurizers.SimpleMoleculeMolGraphFeaturizer()

    train_dset = data.MoleculeDataset(train_data, featurizer)
    val_dset = data.MoleculeDataset(val_data, featurizer)
    test_dset = data.MoleculeDataset(test_data, featurizer)

    train_loader = data.MolGraphDataLoader(train_dset, num_workers=num_workers)
    val_loader = data.MolGraphDataLoader(val_dset, num_workers=num_workers, shuffle=False)
    test_loader = data.MolGraphDataLoader(test_dset, num_workers=num_workers, shuffle=False)

    return train_loader, val_loader, test_loader

train_loader_1, val_loader_1, test_loader_1 = create_data_loaders(train_1, val_1, test_1, num_workers)
train_loader_2, val_loader_2, test_loader_2 = create_data_loaders(train_2, val_2, test_2, num_workers)
train_loader_3, val_loader_3, test_loader_3 = create_data_loaders(train_3, val_3, test_3, num_workers)
train_loader_4, val_loader_4, test_loader_4 = create_data_loaders(train_4, val_4, test_4, num_workers)
train_loader_5, val_loader_5, test_loader_5 = create_data_loaders(train_5, val_5, test_5, num_workers)
train_loader_6, val_loader_6, test_loader_6 = create_data_loaders(train_6, val_6, test_6, num_workers)
train_loader_7, val_loader_7, test_loader_7 = create_data_loaders(train_7, val_7, test_7, num_workers)
train_loader_8, val_loader_8, test_loader_8 = create_data_loaders(train_8, val_8, test_8, num_workers)

In [22]:
def create_model(metric: str, agg: str = 'mean'):
    mp = nn.BondMessagePassing(
    depth = 6,
    d_h = 2200,
    dropout=0.05,
    # activation='SILU'
)
    if agg == 'mean':
        agg = nn.MeanAggregation(

        )

    elif agg == 'attentive':
        agg = nn.AttentiveAggregation(
            output_size=2200,
        )

    elif agg == 'sum': 
        agg = nn.SumAggregation(
        )
    else:
        raise ValueError(f'Invalid aggregation: {agg}')

    # agg = nn.AttentiveAggregation(
    #     output_size=2200,
    # )

    ffn = nn.RegressionFFN(
        input_dim=2200,
        n_layers=6,
        hidden_dim=2200,
        dropout=0.05,
        # activation='SILU',
        # loc=scaler.mean_, # pass in the mean of the training targets
        # scale=scaler.scale_,
        n_tasks=1801 # pass in the scale of the training targets
    )
    batch_norm=False
    if metric == 'rmse':
        metric_list = [nn.RMSEMetric()]
    elif metric == 'sid':
        metric_list = [nn.SIDMetric(), nn.RMSEMetric()]

    else:
        raise ValueError(f'Invalid metric: {metric}')
    return models.MPNN(mp, agg, ffn, batch_norm, metrics=metric_list)

mpnn1 = create_model('rmse', 'attentive')
mpnn2 = create_model('rmse', 'attentive')
mpnn3 = create_model('rmse', 'attentive')
mpnn4 = create_model('rmse', 'attentive')
mpnn5 = create_model('rmse', 'attentive')
mpnn6 = create_model('rmse', 'attentive')
mpnn7 = create_model('rmse', 'attentive')
mpnn8 = create_model('rmse', 'attentive')

In [23]:
trainer_1 = pl.Trainer(
    precision='bf16-mixed',
    logger=True,
    enable_checkpointing=False, # Use `True` if you want to save model checkpoints. The checkpoints will be saved in the `checkpoints` folder.
    enable_progress_bar=True,
    accelerator="auto",
    devices=1,
    max_epochs=6, # number of epochs to train for
    # callbacks=[metric_tracker, early_stop_callback]
)

trainer_2 = pl.Trainer(
    precision='bf16-mixed',
    logger=True,
    enable_checkpointing=False, # Use `True` if you want to save model checkpoints. The checkpoints will be saved in the `checkpoints` folder.
    enable_progress_bar=True,
    accelerator="auto",
    devices=1,
    max_epochs=6, # number of epochs to train for
)

trainer_3 = pl.Trainer(
    precision='bf16-mixed',
    logger=True,
    enable_checkpointing=False, # Use `True` if you want to save model checkpoints. The checkpoints will be saved in the `checkpoints` folder.
    enable_progress_bar=True,
    accelerator="auto",
    devices=1,
    max_epochs=6, # number of epochs to train for
    # callbacks=[metric_tracker, early_stop_callback]
)

trainer_4 = pl.Trainer(
    precision='bf16-mixed',
    logger=True,
    enable_checkpointing=False, # Use `True` if you want to save model checkpoints. The checkpoints will be saved in the `checkpoints` folder.
    enable_progress_bar=True,
    accelerator="auto",
    devices=1,
    max_epochs=6, # number of epochs to train for
)

trainer_5 = pl.Trainer(
    precision='bf16-mixed',
    logger=True,
    enable_checkpointing=False, # Use `True` if you want to save model checkpoints. The checkpoints will be saved in the `checkpoints` folder.
    enable_progress_bar=True,
    accelerator="auto",
    devices=1,
    max_epochs=6, # number of epochs to train for
    # callbacks=[metric_tracker, early_stop_callback]
)

trainer_6 = pl.Trainer(
    precision='bf16-mixed',
    logger=True,
    enable_checkpointing=False, # Use `True` if you want to save model checkpoints. The checkpoints will be saved in the `checkpoints` folder.
    enable_progress_bar=True,
    accelerator="auto",
    devices=1,
    max_epochs=6, # number of epochs to train for
)

trainer_7 = pl.Trainer(
    precision='bf16-mixed',
    logger=True,
    enable_checkpointing=False, # Use `True` if you want to save model checkpoints. The checkpoints will be saved in the `checkpoints` folder.
    enable_progress_bar=True,
    accelerator="auto",
    devices=1,
    max_epochs=6, # number of epochs to train for
)

trainer_8 = pl.Trainer(
    precision='bf16-mixed',
    logger=True,
    enable_checkpointing=False, # Use `True` if you want to save model checkpoints. The checkpoints will be saved in the `checkpoints` folder.
    enable_progress_bar=True,
    accelerator="auto",
    devices=1,
    max_epochs=6, # number of epochs to train for
)

Using bfloat16 Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Using bfloat16 Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Using bfloat16 Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Using bfloat16 Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Using bfloat16 Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Using

In [24]:
trainer_1.fit(mpnn1, train_loader_1, val_loader_1)
trainer_2.fit(mpnn2, train_loader_2, val_loader_2)
trainer_3.fit(mpnn3, train_loader_3, val_loader_3)
trainer_4.fit(mpnn4, train_loader_4, val_loader_4)
trainer_5.fit(mpnn5, train_loader_5, val_loader_5)
trainer_6.fit(mpnn6, train_loader_6, val_loader_6)
trainer_7.fit(mpnn7, train_loader_7, val_loader_7)
trainer_8.fit(mpnn8, train_loader_8, val_loader_8)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loading `train_dataloader` to estimate number of stepping batches.
c:\Users\rhys-\miniconda3\envs\cp3\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.

  | Name            | Type                 | Params
---------------------------------------------------------
0 | message_passing | BondMessagePassing   | 10.3 M
1 | agg             | AttentiveAggregation | 2.2 K 
2 | bn              | Identity             | 0     
3 | predictor       | RegressionFFN        | 33.0 M
  | other params    | n/a                  | 1.8 K 
---------------------------------------------------------
43.3 M    Trainable params
1.8 K     Non-trainable params
43.3 M    Total params
173.278   Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

c:\Users\rhys-\miniconda3\envs\cp3\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Epoch 5: 100%|██████████| 1283/1283 [01:16<00:00, 16.87it/s, v_num=11, train/loss=0.00476, val_loss=0.0684]

`Trainer.fit` stopped: `max_epochs=6` reached.


...

Epoch 5: 100%|██████████| 1283/1283 [01:20<00:00, 16.02it/s, v_num=17, train/loss=0.00579, val_loss=0.0688]


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loading `train_dataloader` to estimate number of stepping batches.

  | Name            | Type                 | Params
---------------------------------------------------------
0 | message_passing | BondMessagePassing   | 10.3 M
1 | agg             | AttentiveAggregation | 2.2 K 
2 | bn              | Identity             | 0     
3 | predictor       | RegressionFFN        | 33.0 M
  | other params    | n/a                  | 1.8 K 
---------------------------------------------------------
43.3 M    Trainable params
1.8 K     Non-trainable params
43.3 M    Total params
173.278   Total estimated model params size (MB)


Epoch 5: 100%|██████████| 1283/1283 [01:19<00:00, 16.08it/s, v_num=18, train/loss=0.00584, val_loss=0.0689]

`Trainer.fit` stopped: `max_epochs=6` reached.


Epoch 5: 100%|██████████| 1283/1283 [01:19<00:00, 16.08it/s, v_num=18, train/loss=0.00584, val_loss=0.0689]


In [25]:
from chemprop import models 

trainer_1.save_checkpoint(r'C:\Users\rhys-\OneDrive\Documents\Github\thesis_results\models\xtb_mpnn1.ckpt')
trainer_2.save_checkpoint(r'C:\Users\rhys-\OneDrive\Documents\Github\thesis_results\models\xtb_mpnn2.ckpt')
trainer_3.save_checkpoint(r'C:\Users\rhys-\OneDrive\Documents\Github\thesis_results\models\xtb_mpnn3.ckpt')
trainer_4.save_checkpoint(r'C:\Users\rhys-\OneDrive\Documents\Github\thesis_results\models\xtb_mpnn4.ckpt')
trainer_5.save_checkpoint(r'C:\Users\rhys-\OneDrive\Documents\Github\thesis_results\models\xtb_mpnn5.ckpt')
trainer_6.save_checkpoint(r'C:\Users\rhys-\OneDrive\Documents\Github\thesis_results\models\xtb_mpnn6.ckpt')
trainer_7.save_checkpoint(r'C:\Users\rhys-\OneDrive\Documents\Github\thesis_results\models\xtb_mpnn7.ckpt')
trainer_8.save_checkpoint(r'C:\Users\rhys-\OneDrive\Documents\Github\thesis_results\models\xtb_mpnn8.ckpt')

checkpoint = torch.load(r"C:\Users\rhys-\OneDrive\Documents\Github\thesis_results\models\xtb_mpnn1.ckpt")
checkpoint2 = torch.load(r"C:\Users\rhys-\OneDrive\Documents\Github\thesis_results\models\xtb_mpnn2.ckpt")
checkpoint3 = torch.load(r"C:\Users\rhys-\OneDrive\Documents\Github\thesis_results\models\xtb_mpnn3.ckpt")
checkpoint4 = torch.load(r"C:\Users\rhys-\OneDrive\Documents\Github\thesis_results\models\xtb_mpnn4.ckpt")
checkpoint5 = torch.load(r"C:\Users\rhys-\OneDrive\Documents\Github\thesis_results\models\xtb_mpnn5.ckpt")
checkpoint6 = torch.load(r"C:\Users\rhys-\OneDrive\Documents\Github\thesis_results\models\xtb_mpnn6.ckpt")
checkpoint7 = torch.load(r"C:\Users\rhys-\OneDrive\Documents\Github\thesis_results\models\xtb_mpnn7.ckpt")
checkpoint8 = torch.load(r"C:\Users\rhys-\OneDrive\Documents\Github\thesis_results\models\xtb_mpnn8.ckpt")

# Assuming 'output_size' needs to be added or corrected
# You need to know the correct value for 'output_size'
checkpoint['hyper_parameters']['agg']['output_size'] = 2200
checkpoint2['hyper_parameters']['agg']['output_size'] = 2200
checkpoint3['hyper_parameters']['agg']['output_size'] = 2200
checkpoint4['hyper_parameters']['agg']['output_size'] = 2200
checkpoint5['hyper_parameters']['agg']['output_size'] = 2200
checkpoint6['hyper_parameters']['agg']['output_size'] = 2200
checkpoint7['hyper_parameters']['agg']['output_size'] = 2200
checkpoint8['hyper_parameters']['agg']['output_size'] = 2200

torch.save(checkpoint, r"C:\Users\rhys-\OneDrive\Documents\Github\thesis_results\models\xtb_modified_mpnn_1.ckpt")
torch.save(checkpoint2, r"C:\Users\rhys-\OneDrive\Documents\Github\thesis_results\models\xtb_modified_mpnn_2.ckpt")
torch.save(checkpoint3, r"C:\Users\rhys-\OneDrive\Documents\Github\thesis_results\models\xtb_modified_mpnn_3.ckpt")
torch.save(checkpoint4, r"C:\Users\rhys-\OneDrive\Documents\Github\thesis_results\models\xtb_modified_mpnn_4.ckpt")
torch.save(checkpoint5, r"C:\Users\rhys-\OneDrive\Documents\Github\thesis_results\models\xtb_modified_mpnn_5.ckpt")
torch.save(checkpoint6, r"C:\Users\rhys-\OneDrive\Documents\Github\thesis_results\models\xtb_modified_mpnn_6.ckpt")
torch.save(checkpoint7, r"C:\Users\rhys-\OneDrive\Documents\Github\thesis_results\models\xtb_modified_mpnn_7.ckpt")
torch.save(checkpoint8, r"C:\Users\rhys-\OneDrive\Documents\Github\thesis_results\models\xtb_modified_mpnn_8.ckpt")



In [26]:
loaded_model = models.MPNN.load_from_checkpoint(r"C:\Users\rhys-\OneDrive\Documents\Github\thesis_results\models\xtb_modified_mpnn_1.ckpt")


In [27]:
trainer_loaded = pl.Trainer(
    precision='bf16-mixed',
    logger=True,
    enable_checkpointing=False, # Use `True` if you want to save model checkpoints. The checkpoints will be saved in the `checkpoints` folder.
    enable_progress_bar=True,
    accelerator="auto",
    devices=1,
    max_epochs=6, # number of epochs to train for
    # callbacks=[metric_tracker, early_stop_callback]
)

trainer_loaded.test(loaded_model, test_loader_1)

Using bfloat16 Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
c:\Users\rhys-\miniconda3\envs\cp3\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Testing DataLoader 0: 100%|██████████| 214/214 [00:10<00:00, 19.89it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test/rmse           0.06847275793552399
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test/rmse': 0.06847275793552399}]